In [1]:
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn import metrics
import tensorflow as tf
import os
import pickle
from tqdm.notebook import tqdm
from collections import defaultdict, OrderedDict
import requests

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [3]:
PATH = "/Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/"

In [4]:
listdir = os.listdir(os.path.join(PATH, "parsed_content"))

In [5]:
titles = {}
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[1].split(".")[0]
        with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
            titles[docid] = pickle.load(f)[1]
    except:
        bad_files.append(file)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-9da8ada4c0c7>", line 3, in <module>
    for file in tqdm(listdir, total=len(listdir)):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/notebook.py", line 234, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/std.py", line 1184, in __iter__
    self.refresh(lock_args=self.lock_args)
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/std.py", line 1380, in refresh
    self.display()
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/notebook.py", line 152, in display
    pbar.value = self.n
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/traitlets/traitlets.py"

KeyboardInterrupt: 

In [ ]:
bad_files

In [ ]:
len(titles)

In [16]:
bad_parsed = {}
for file in tqdm(bad_files, total=len(bad_files)):
    docid = file.split("_")[1].split(".")[0]
    with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
        bad_parsed[docid] = pickle.load(f)[1]

In [17]:
bad_parsed

{}

In [18]:
for key, value in bad_parsed.items():
    titles[key] = value

In [37]:
len(titles)

38114

In [38]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")

In [6]:
with open(PATH + "sample.technosphere.ir1.textrelevance.submission.txt", "r") as f:
    sample = f.read().split("\n")

In [7]:
query_docid = defaultdict(list)
for item in sample[1:]:
    if item:
        qid = item.split(",")[0]
        docid = item.split(",")[1]
        query_docid[qid].append(docid)

In [41]:
queries_to_map = []

for query in queries[:-1]:
    queries_to_map.append(query.split("\t")[1])

In [42]:
def spellcheck(text):
    payload = {"text": text}
    response = requests.get("https://speller.yandex.net/services/spellservice.json/checkText",
                            params=payload).json()
    
    fixed_text = text
    for item in response:
        fixed_text = fixed_text.replace(item["word"], item["s"][0])
    return fixed_text

In [25]:
fix_queries_to_map = [spellcheck(query) for query in tqdm(queries_to_map, total=len(queries_to_map))]

In [32]:
queries_embeddings = embed(fix_queries_to_map)

In [33]:
import os

In [34]:
with open(os.path.join(PATH, "queries_embeddings"), "wb") as f:
    pickle.dump(queries_embeddings, f)

In [36]:
import sys

In [49]:
titles_list = []
for i in range(1, len(titles) + 1):
    titles_list.append(titles[str(i)])

In [50]:
len(titles_list)

38114

In [15]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/title_embeddings

In [51]:
with open(os.path.join(PATH, "titles.pkl"), "wb") as f:
    pickle.dump(titles, f)

In [8]:
with open(os.path.join(PATH, "titles.pkl"), "rb") as f:
    titles = pickle.load(f)

In [16]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        titles_tmp = [titles[docid][:500] for docid in docids]
        titles_emb_tmp = embed(titles_tmp)
        with open(os.path.join(PATH, "title_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(titles_emb_tmp, f)
    except:
        print(query)
        bad_queries.append(query)

In [17]:
listdir = os.listdir(os.path.join(PATH, "title_embeddings"))

In [18]:
len(listdir)

399

In [52]:
with open(os.path.join(PATH, "title_embeddings/1.pkl"), "rb") as f:
    data = pickle.load(f)

In [20]:
with open(os.path.join(PATH, "queries_embeddings"), "rb") as f:
    queries_embeddings = pickle.load(f)

In [36]:
np.argsort(metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[0], (1, -1)), data)[0])

array([27, 77, 52, 73, 65, 57,  6, 74, 66, 56, 50, 84, 59, 43, 69, 16, 93,
       53,  2, 55, 82, 87, 95, 67, 71, 89, 81, 23, 46, 76, 72, 60, 61, 90,
       17, 63, 94, 10, 45, 21,  0, 38, 25, 13, 75, 44, 33, 83,  8, 11, 36,
       12, 34, 79, 64, 47, 92,  1, 40, 32, 30, 41, 35, 42, 70, 15, 58, 54,
       51, 86, 26, 78, 14,  7, 28, 31,  3, 62, 24, 20, 37, 29, 88, 22, 18,
       19, 48,  9, 68, 80, 49, 91,  5, 39,  4, 85])

In [60]:
np.array(query_docid["1"])[np.argsort(metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data)[0])]

array(['28', '78', '53', '74', '66', '58', '7', '75', '67', '57', '51',
       '85', '60', '44', '70', '17', '94', '54', '3', '56', '83', '88',
       '96', '68', '72', '90', '82', '24', '47', '77', '73', '61', '62',
       '91', '18', '64', '95', '11', '46', '22', '1', '39', '26', '14',
       '76', '45', '34', '84', '9', '12', '37', '13', '35', '80', '65',
       '48', '93', '2', '41', '33', '31', '42', '36', '43', '71', '16',
       '59', '55', '52', '87', '27', '79', '15', '8', '29', '32', '4',
       '63', '25', '21', '38', '30', '89', '23', '19', '20', '49', '10',
       '69', '81', '50', '92', '6', '40', '5', '86'], dtype='<U2')

In [61]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f:
        data = pickle.load(f)
    ranged_query_docid[query] = np.array(docids)[np.argsort(metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data)[0])]

In [62]:
ranged_query_docid

{'1': array(['28', '78', '53', '74', '66', '58', '7', '75', '67', '57', '51',
        '85', '60', '44', '70', '17', '94', '54', '3', '56', '83', '88',
        '96', '68', '72', '90', '82', '24', '47', '77', '73', '61', '62',
        '91', '18', '64', '95', '11', '46', '22', '1', '39', '26', '14',
        '76', '45', '34', '84', '9', '12', '37', '13', '35', '80', '65',
        '48', '93', '2', '41', '33', '31', '42', '36', '43', '71', '16',
        '59', '55', '52', '87', '27', '79', '15', '8', '29', '32', '4',
        '63', '25', '21', '38', '30', '89', '23', '19', '20', '49', '10',
        '69', '81', '50', '92', '6', '40', '5', '86'], dtype='<U2'),
 '2': array(['154', '125', '153', '192', '165', '149', '131', '103', '99',
        '156', '150', '166', '186', '185', '179', '137', '101', '114',
        '183', '141', '177', '102', '167', '164', '180', '171', '116',
        '145', '181', '133', '152', '159', '118', '182', '155', '143',
        '100', '135', '129', '190', '147', '157', '14

In [63]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [64]:
ranged_query_docid

{'1': ['28',
  '78',
  '53',
  '74',
  '66',
  '58',
  '7',
  '75',
  '67',
  '57',
  '51',
  '85',
  '60',
  '44',
  '70',
  '17',
  '94',
  '54',
  '3',
  '56',
  '83',
  '88',
  '96',
  '68',
  '72',
  '90',
  '82',
  '24',
  '47',
  '77',
  '73',
  '61',
  '62',
  '91',
  '18',
  '64',
  '95',
  '11',
  '46',
  '22',
  '1',
  '39',
  '26',
  '14',
  '76',
  '45',
  '34',
  '84',
  '9',
  '12',
  '37',
  '13',
  '35',
  '80',
  '65',
  '48',
  '93',
  '2',
  '41',
  '33',
  '31',
  '42',
  '36',
  '43',
  '71',
  '16',
  '59',
  '55',
  '52',
  '87',
  '27',
  '79',
  '15',
  '8',
  '29',
  '32',
  '4',
  '63',
  '25',
  '21',
  '38',
  '30',
  '89',
  '23',
  '19',
  '20',
  '49',
  '10',
  '69',
  '81',
  '50',
  '92',
  '6',
  '40',
  '5',
  '86'],
 '2': ['154',
  '125',
  '153',
  '192',
  '165',
  '149',
  '131',
  '103',
  '99',
  '156',
  '150',
  '166',
  '186',
  '185',
  '179',
  '137',
  '101',
  '114',
  '183',
  '141',
  '177',
  '102',
  '167',
  '164',
  '180',
  '171

In [65]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_09_2.txt", "w") as f:
    f.write("\n".join(submit))

In [66]:
len(submit)

38773

In [67]:
counter = 0
for query in ranged_query_docid:
    counter += len(ranged_query_docid[query])
print(counter)

38772


# Add meta tags

In [110]:
listdir = os.listdir(os.path.join(PATH, "parsed_content"))

In [73]:
meta = {}
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[1].split(".")[0]
        with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
            meta[docid] = pickle.load(f)[2]
    except:
        bad_files.append(file)

In [75]:
len(meta)

38114

In [77]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/meta_embeddings

In [78]:
with open(os.path.join(PATH, "meta.pkl"), "wb") as f:
    pickle.dump(meta, f)

In [111]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        meta_tmp = [meta[docid][:1000] for docid in docids]
        meta_emb_tmp = embed(meta_tmp)
        with open(os.path.join(PATH, "meta_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(meta_emb_tmp, f)
    except:
        print(query)
        bad_queries.append(query)

In [112]:
bad_queries

[]

In [113]:
len(os.listdir(os.path.join(PATH, "meta_embeddings")))

399

In [88]:
bad_queries = ["6", "8"]
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    if query in bad_queries:
#     try:
        meta_tmp = [meta[docid][:500] for docid in docids]
        meta_emb_tmp = embed(meta_tmp)
        with open(os.path.join(PATH, "meta_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(meta_emb_tmp, f)
#     except:
#         print(query)
#         bad_queries.append(query)

In [114]:
len(os.listdir(os.path.join(PATH, "meta_embeddings")))

399

In [115]:
with open(os.path.join(PATH, "meta_embeddings", f"{6}.pkl"), "rb") as f:
    data = pickle.load(f)

In [116]:
data.shape

TensorShape([108, 512])

In [119]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f:
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"meta_embeddings/{query}.pkl"), "rb") as f:
        data_meta = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    meta_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_meta)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * meta_score)]

In [120]:
ranged_query_docid

{'1': array(['74', '53', '28', '78', '58', '51', '75', '70', '17', '90', '66',
        '3', '67', '82', '60', '88', '85', '73', '68', '56', '54', '96',
        '24', '77', '7', '47', '57', '61', '44', '91', '62', '64', '18',
        '22', '95', '46', '94', '83', '72', '11', '26', '35', '76', '34',
        '36', '1', '39', '93', '42', '37', '65', '12', '31', '48', '14',
        '16', '45', '71', '84', '9', '52', '59', '2', '41', '33', '13',
        '80', '27', '87', '8', '43', '29', '15', '55', '32', '25', '63',
        '4', '79', '19', '30', '49', '21', '38', '89', '69', '23', '92',
        '20', '6', '81', '5', '10', '40', '50', '86'], dtype='<U2'),
 '2': array(['125', '154', '149', '192', '131', '103', '156', '150', '153',
        '99', '185', '186', '165', '114', '179', '183', '101', '141',
        '137', '166', '145', '102', '116', '182', '159', '177', '164',
        '180', '100', '167', '118', '171', '157', '181', '155', '133',
        '146', '147', '117', '112', '130', '152', '14

In [121]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [122]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_09_3.txt", "w") as f:
    f.write("\n".join(submit))

In [109]:
counter = 0
for index, item in enumerate(meta.values()):
    counter += len(item) > 1000
print(counter)

458


In [123]:
from string import punctuation

In [125]:
import re

In [157]:
with open(os.path.join(PATH, "parsed_content/parsed_26.pkl"), "rb") as f:
    data = pickle.load(f)[2]

In [158]:
data

'Каталог путешествий RUSSIANTRAVELLER.RU. Новости туризма. Каталог компаний: турагентства, туроператоры, отели, визовые центры, финансовые компании. Справочная информация, адреса посольств, бронирование готиницы горящие туры, Русский путещественник, туристический справочник, индивидуальный отдых, отпуск, автобусные туры, поездка, горящие круизы, путешествие, авиабилеты'

In [183]:
re.sub(r'[!#$%&()*+<=>?@\[\\\]^_`{|}~\n\t\r]', " ", data + "\n" + "~[]")

'Каталог путешествий RUSSIANTRAVELLER.RU. Новости туризма. Каталог компаний: турагентства, туроператоры, отели, визовые центры, финансовые компании. Справочная информация, адреса посольств, бронирование готиницы горящие туры, Русский путещественник, туристический справочник, индивидуальный отдых, отпуск, автобусные туры, поездка, горящие круизы, путешествие, авиабилеты    '

In [182]:
re.sub('[.]', ";", data + "\n" + "~[]")

'Каталог путешествий RUSSIANTRAVELLER;RU; Новости туризма; Каталог компаний: турагентства, туроператоры, отели, визовые центры, финансовые компании; Справочная информация, адреса посольств, бронирование готиницы горящие туры, Русский путещественник, туристический справочник, индивидуальный отдых, отпуск, автобусные туры, поездка, горящие круизы, путешествие, авиабилеты\n~[]'

In [171]:
"f \n sfddsdf   \n\r\t".strip()

'f \n sfddsdf'

In [188]:
re.sub('\xa0', " ", "\xa0 sdfasdf \n ---- \xa0")

'  sdfasdf \n ----  '